<h1 align ="center"> REST API In Context Learning Samples</h1>
<hr>
   
# Chat Completions

In [1]:
import json
import os
import requests
import base64

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup search_service_endpoint, search_index_name, search_query_key, deployment_name, openai_api_base, openai_api_key and openai_api_version.

In [3]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the Azure Search service endpoint.e.g. https://<your search service name>.search.windows.net  
search_service_endpoint = config_details['AZURE_SEARCH_SERVICE_ENDPOINT']

# Setting up the Azure Search service index
search_index_name = config_details['AZURE_SEARCH_INDEX_NAME']

# Setting up the Azure Search service query key
search_query_key = os.getenv("AZURE_SEARCH_QUERY_KEY")

# Setting up the deployment name
deployment_name = config_details['GPT-4V_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

In [4]:
# Create Azure Search index (the link will be updated once it goes public)
# https://github.com/zhizhoualan/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-image-index-creation-python-sample.ipynb

# System messages and user prompt
sys_message = "You are an AI assistant that helps people find information."
user_prompt = "What are the types of the apple(s) shown in this image?"

# Encode the image in base64
image_file_path = "../../common/images/test_Gala.jpeg"  # Another example including two apples: "../../common/images/two_apples.jpeg".
with open(image_file_path, 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('ascii')
    
# Construct the API request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/extensions/chat/completions?api-version={openai_api_version}"

# Including the api-key in HTTP headers
headers = {
    "Content-Type": "application/json",
    "api-key": openai_api_key,
}

# Payload for the request
payload = {
  "dataSources": [
    {
      "type": "AzureCognitiveSearch",
      "parameters": {
        "endpoint": search_service_endpoint,
        "key": search_query_key,
        "indexName": search_index_name
      }
    }
  ],
  "messages": [
    {
      "role": "system",
      "content": [
        sys_message
      ]
    },
    {
      "role": "user",
      "content": [
        user_prompt, # Pass the prompt
        {
          "image": encoded_image  #Pass the encoded image
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

# Send the request and handle the response
try:
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad HTTP status codes
    response_content = response.json()
    print(response_content['choices'][0]['message']['content'])
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

It's a Gala apple.
